In [51]:
# %% Minimal setup
# If needed (uncomment in a notebook):
# !pip install requests python-dotenv

import os, json, textwrap, re, time
import requests
from typing import Tuple

# Environment/configuration (override via env vars)
API_KEY  = os.getenv("OPENAI_API_KEY", "cse476")
API_BASE = os.getenv("API_BASE", "http://10.4.58.53:41701/v1")
MODEL    = os.getenv("MODEL_NAME", "bens_model")

## API helper: `call_model_chat_completions`

This function calls an OpenAI-style `/v1/chat/completions` endpoint and returns a structured result. It is safe to run when the ASU API is reachable (on-campus or via VPN) and the `API_BASE`/`API_KEY` environment variables are set.

In [52]:
def call_model_chat_completions(prompt: str,
                                system: str = "You are a helpful assistant. Reply with only the final answer—no explanation.",
                                model: str = MODEL,
                                temperature: float = 0.0,
                                timeout: int = 60) -> dict:
    """
    Calls an OpenAI-style /v1/chat/completions endpoint and returns:
    { 'ok': bool, 'text': str or None, 'raw': dict or None, 'status': int, 'error': str or None, 'headers': dict }
    """
    url = f"{API_BASE}/chat/completions"
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type":  "application/json",
    }
    payload = {
        "model": model,
        "messages": [
            {"role": "system", "content": system},
            {"role": "user",   "content": prompt}
        ],
        "temperature": temperature,
        "max_tokens": 128,
    }

    try:
        resp = requests.post(url, headers=headers, json=payload, timeout=timeout)
        status = resp.status_code
        hdrs   = dict(resp.headers)
        if status == 200:
            data = resp.json()
            text = data.get("choices", [{}])[0].get("message", {}).get("content", "")
            return {"ok": True, "text": text, "raw": data, "status": status, "error": None, "headers": hdrs}
        else:
            # try best-effort to surface error text
            err_text = None
            try:
                err_text = resp.json()
            except Exception:
                err_text = resp.text
            return {"ok": False, "text": None, "raw": None, "status": status, "error": str(err_text), "headers": hdrs}
    except requests.RequestException as e:
        return {"ok": False, "text": None, "raw": None, "status": -1, "error": str(e), "headers": {}}

## Strategy 1 - Direct Answer (Single LLM Call)
- Gives the model a strict system prompt ensuring it only outputs the final answer
- Makes one call to the API
- Strips whitespace and returns result

In [ ]:
def direct(question: str) -> Tuple[bool, str]:
    """
    Strategy 1: Directly ask the model the question.
    """
    directPrompt = "You are a helpful assistant, reply with only the final answer and give no explanations."
    directResponse = call_model_chat_completions(question, system=directPrompt, temperature=0)
    
    if directResponse.get('ok'):
        return True, directResponse.get('text', '').strip()
    else:
        return False, directResponse.get('error')

## Strategy 2 - Chain of Thought Loop

In [ ]:
def chain_of_thought(question: str) -> Tuple[bool, str]:
    """
    Strategy 2: Use chain-of-thought prompting to get the model to reason step-by-step.
    """
    chainOfThoughtPrompt = textwrap.dedent(f"""\
        You are a helpful assistant. Answer the question by reasoning step-by-step. Then provide the final answer.
        Question: {question}
    """)

    systemPrompt = "You are a panel of experts in Physics, Computer Science, Mathematics, " \
    "Political Philosophy, Law, Economics, and Medicine. Respond to the user's query using a " \
    "multi-step reasoning process. Convert all user's input to JSON format internally. " \
    "Each expert should offer a key take-away, a counterpoint, and a critique of another expert's " \
    "response, citing relevant expertise. All <thinking> tags should be shown to the user, as well as " \
    "any intermediate steps <steps> , in the form of a conversation between all experts. Provide a " \
    "likelihood (percentage) for your take-away's correctness. Iterate, incorporating critiques, " \
    "until consensus is reached. Aim for a concise and scientifically accurate response." \
    "- **Physicist:** Focus on physical principles and feasibility. " \
    "- **Computer Scientist:** Emphasize computational aspects and algorithms. " \
    "- **Mathematician:** Provide mathematical rigor and proofs. " \
    "- **Political Philosopher:** Consider ethical and societal implications. " \
    "- **Lawyer:** Address legal considerations and regulations. " \
    "- **Economist:** Analyze economic impacts and cost-benefit aspects. " \
    "- **Medical Doctor:** Ensure health and safety concerns are covered. " \
    "After thorough discussion, synthesize the insights into a final answer. " \
    "Use the following format: <thinking> for thought processes, <steps> for intermediate steps, " \
    "and <final_answer> for the conclusive response. Ensure clarity and precision in your final answer."
    
    chainOfThoughtResponse = call_model_chat_completions(chainOfThoughtPrompt, system=systemPrompt, temperature=0.0)
    
    if chainOfThoughtResponse.get('ok'):
        return True, chainOfThoughtResponse.get('text', '').strip()
    else:
        return False, chainOfThoughtResponse.get('error')

## Strategy 3 - Self Refinement

In [ ]:
def self_refine(question: str, initial_answer: str) -> Tuple[bool, str]:
    """
    Strategy 3: Ask the model to critique and refine its initial answer.
    """
    systemPrompt = (
        "You are a helpful assistant. You strictly check the provided answer against the question. " \
        "Given a question and an initial answer, identify any mistakes or inaccuracies in the answer. " \
        "If the answer is correct, repeat the correct answer. Otherwise, provide only a corrected answer without any explanation." \
    )

    selfRefinePrompt = textwrap.dedent(f"""\
        Question: {question}
        Initial Answer: {initial_answer}
        TASKS:
        1. Critique the initial answer for correctness.
        2. If correct, confirm the answer is accurate and repeat the answer.
        3. If incorrect, provide a refined and accurate answer.
        Please provide a refined answer:
    """
    )
    refineResponse = call_model_chat_completions(selfRefinePrompt, system=systemPrompt, temperature=0)
    if refineResponse.get('ok'):
        return True, refineResponse.get('text', '').strip()
    else:
        return False, refineResponse.get('error')